Cerebras GPT

In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("cerebras/Cerebras-GPT-111M")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained("cerebras/Cerebras-GPT-111M")

In [2]:
# mps device
device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
device

device(type='mps')

In [3]:
# model = model.to(device)
model.device

device(type='cpu')

In [4]:
# prompt = 'How many enterprise Tactics IDs are in the Mitre Att&ck v13 matrix released in April 2023?\n\n Assistant:'
# prompt = """ 
# Explain the following payload. Payload is GET /jenkins/postnuke/index.php?module=My_eGallery&do=showpic&pid=-1/**/AND/**/1=2/**/UNION/**/ALL/**/SELECT/**/0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,concat(0x3C7230783E,pn_uname,0x3a,pn_pass,0x3C7230783E),0,0,0/**/FROM/**/md_users/**/WHERE/**/pn_uid=$id/* HTTP/1.1 Content-Length: 36 Content-Type: application/x-www-form-urlencoded Host: www.test.go.kr Connection: Keep-Alive User-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None).\n\nAssistant:
# """
prompt = 'User: What is xss attack?\n\nAssistant: '

In [5]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generated_text = pipe(prompt, max_length=256, do_sample=False, no_repeat_ngram_size=2)[0]
print(generated_text['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: What is xss attack?

Assistant: 
   - I want to know what is the problem.
- I have a question about the xs attack.  I don't know how to solve it. I know I can't find a solution. But I'm not sure how I could solve this problem, I just want it to work.




Pytorch based inference

In [6]:
# Encode the prompt using the tokenizer
input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [7]:
# Generate text based on the encoded prompt
with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        do_sample=True,
        top_p=0.75,
        top_k=85,
        temperature=1.99,
        typical_p=1,
        repetition_penalty=1.3,
        max_length=250,  # The maximum number of tokens to generate
        num_beams=5,    # The number of beams to use for beam search
#         early_stopping=True,  # Stop generation when the model predicts an end-of-sequence token
    )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [8]:
# Decode the generated text and print it
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

User: What is xss attack?

Assistant:  I believe it is Xss attack.  If you are in a position of 0 or 1 you are in a position of 3 or even more then the attack happens.

A:

You should use the Xss attacks as well, since Xss attacks do not apply in all cases, and you are only using the Xss attacks on Xss attacks as well.
Here is my answer:

XSS attacks can be implemented by using XSS attacks as well.
XSS attacks are often used in XSS attack scenarios for XSS attacks. You should not use XSS attacks.
XSS attacks are designed to be used in different environments.
However, they can be very useful in different situations.

A:

XSS attacks have an advantage over Xss attacks.
The reason that XSS attacks are quite powerful depends on your Xss attack.  There are several ways to implement XSS attacks, so you are not trying to use Xss attacks.  The first thing to try is to use XSS attacks, which makes it really hard to learn from Xss attacks.  You could also use Xss


Pytorch based fine tuning

In [9]:
import datasets
dataset = datasets.load_dataset('json', data_files='chat_gpt_context/security_base_sample.json', field='train')

Using custom data configuration default-35b97c11878f9c91


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 659.69it/s]


Dataset json downloaded and prepared to /Users/choiwb/.cache/huggingface/datasets/json/default-35b97c11878f9c91/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 842.57it/s]


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 43
    })
})

In [11]:
cutoff_len = 1024

def generate_prompt(entry):
    if entry['input']:
        return f"User: {entry['instruction']}: {entry['input']}\n\nAssistant: {entry['output']}"
    else:
        return f"User: {entry['instruction']}\n\nAssistant: {entry['output']}"

def tokenize(item, add_eos_token=True):
    result = tokenizer(
        generate_prompt(item),
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )

    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

In [12]:
train_val = dataset["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_data = train_val["train"].shuffle().map(tokenize)
val_data = train_val["test"].shuffle().map(tokenize)

Parameter 'function'=<function tokenize at 0x2b7b45280> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 5/5 [00:00<00:00, 1330.43ex/s]


In [13]:
'''
if 'model' in globals(): 
    del model
    # torch.cuda.empty_cache()

model = transformers.AutoModelForCausalLM.from_pretrained(
    'cerebras/Cerebras-GPT-111M',    
    
    # load_in_8bit=True,
    # torch_dtype=torch.float16,

    # device_map={'': 0}
    #device = torch.device("cpu")
    # device_map = 'auto'
)
'''

'\nif \'model\' in globals(): \n    del model\n    # torch.cuda.empty_cache()\n\nmodel = transformers.AutoModelForCausalLM.from_pretrained(\n    \'cerebras/Cerebras-GPT-111M\',    \n    \n    # load_in_8bit=True,\n    # torch_dtype=torch.float16,\n\n    # device_map={\'\': 0}\n    #device = torch.device("cpu")\n    # device_map = \'auto\'\n)\n'

In [14]:
'''
import peft

model = peft.prepare_model_for_int8_training(model)

model = peft.get_peft_model(model, peft.LoraConfig(
    r=8,
    lora_alpha=16,
    # target_modules=["q_proj", "v_proj"],
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
))
'''

'\nimport peft\n\nmodel = peft.prepare_model_for_int8_training(model)\n\nmodel = peft.get_peft_model(model, peft.LoraConfig(\n    r=8,\n    lora_alpha=16,\n    # target_modules=["q_proj", "v_proj"],\n    target_modules=["c_attn"],\n    lora_dropout=0.05,\n    bias="none",\n    task_type="CAUSAL_LM",\n))\n'

In [15]:
# import peft

# model = peft.PeftModel.from_pretrained(
#     model,
#     # 'lora-cerebras-gpt2.7b-hh-rlhf-helpful-online',
#     output_dir,
#     torch_dtype=torch.float16
# )

In [16]:
import os
import wandb 

output_dir = 'cerebras-gpt111m-finetune'

use_wandb = True,
wandb_run_name = f"{output_dir}-{wandb.util.generate_id()}"

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]=output_dir

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

In [17]:
training_args = transformers.TrainingArguments(
    # per_device_train_batch_size=16, 
    per_device_train_batch_size=1, 

    gradient_accumulation_steps=8,  
    num_train_epochs=30,  
    learning_rate=1e-4, 
    # only be used on CUDA devices.
    # fp16=True,
    
    optim="adamw_torch",
    logging_steps=10, 
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    save_steps=200,
    output_dir=output_dir, 
    save_total_limit=3,

    report_to="wandb" if use_wandb else None,
    run_name=wandb_run_name if use_wandb else None,
)

In [18]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_args, 
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=1, return_tensors="pt", padding=True
    ),
)

In [19]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [20]:
model.config.use_cache = False
result = trainer.train()
model.save_pretrained(output_dir)

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: khuam1216 (ngcsm). Use `wandb login --relogin` to force relogin


  8%|▊         | 10/120 [00:18<03:28,  1.90s/it]

{'loss': 2.3706, 'learning_rate': 9.166666666666667e-05, 'epoch': 2.11}


 17%|█▋        | 20/120 [00:37<02:59,  1.79s/it]

{'loss': 0.6996, 'learning_rate': 8.333333333333334e-05, 'epoch': 4.21}


...

{'loss': 0.0763, 'learning_rate': 8.333333333333334e-06, 'epoch': 23.16}


100%|██████████| 120/120 [03:41<00:00,  1.92s/it]

{'loss': 0.0754, 'learning_rate': 0.0, 'epoch': 25.26}
{'train_runtime': 227.8847, 'train_samples_per_second': 5.003, 'train_steps_per_second': 0.527, 'train_loss': 0.34728946536779404, 'epoch': 25.26}


100%|██████████| 120/120 [03:43<00:00,  1.86s/it]


train/epoch,▁▂▂▃▄▄▅▅▆▇▇██
train/global_step,▁▂▂▃▄▄▅▅▆▇▇██
train/learning_rate,█▇▇▆▅▅▄▄▃▂▂▁
train/loss,█▃▂▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,25.26
train/global_step,120


Pytorch based fine tuning model load & inference

In [21]:
model.config
print(model.dtype)

# model.half()
model.eval()

torch.float32


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-9): 10 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): GELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [22]:
# 저장된 모델 호출
output_dir = 'cerebras-gpt111m-finetune'
model = transformers.AutoModelForCausalLM.from_pretrained(output_dir)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-9): 10 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): GELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [23]:
# prompt = 'What is XSS (Cross Site Scripting) attack?'
# prompt = 'How many enterprise Tactics IDs are in the Mitre Att&ck v13 matrix released in April 2023?'
# prompt = """ 
#Explain the following payload. Payload is GET /jenkins/postnuke/index.php?module=My_eGallery&do=showpic&pid=-1/**/AND/**/1=2/**/UNION/**/ALL/**/SELECT/**/0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,concat(0x3C7230783E,pn_uname,0x3a,pn_pass,0x3C7230783E),0,0,0/**/FROM/**/md_users/**/WHERE/**/pn_uid=$id/* HTTP/1.1 Content-Length: 36 Content-Type: application/x-www-form-urlencoded Host: www.test.go.kr Connection: Keep-Alive User-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None).\n\n Assistant:
# """
prompt = """ 
Explain the following payload. Payload is GET /jenkins/postnuke/index.php?module=My_eGallery&do=showpic&pid=-1/**/AND/**/1=2/**/UNION/**/ALL/**/SELECT/**/0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,concat(0x3C7230783E,pn_uname,0x3a,pn_pass,0x3C7230783E),0,0,0/**/FROM/**/md_users/**/WHERE/**/pn_uid=$id/* HTTP/1.1 Content-Length: 36 Content-Type: application/x-www-form-urlencoded Host: www.test.go.kr Connection: Keep-Alive User-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None).
"""
# prompt = 'What is TA0006 (Credential Access)'

In [24]:
inputs = tokenizer(prompt, return_tensors="pt")
# input_ids = inputs["input_ids"].to(model.device)
input_ids = inputs["input_ids"]

In [25]:
generation_config = transformers.GenerationConfig(
    max_new_tokens=100,
    temperature=0.2,
    top_p=0.75,
    top_k=50,
    repetition_penalty=1.2,
    do_sample=True,
    early_stopping=True,
    # num_beams=5,
    
    pad_token_id=model.config.pad_token_id,
    eos_token_id=model.config.eos_token_id,
)

In [26]:
'''
with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config
    )[0].cuda()
'''
with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config
    )[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [27]:
result = tokenizer.decode(output, skip_special_tokens=True).strip()
print(result)

Explain the following payload. Payload is GET /jenkins/postnuke/index.php?module=My_eGallery&do=showpic&pid=-1/**/AND/**/1=2/**/UNION/**/ALL/**/SELECT/**/0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,concat(0x3C7230783E,pn_uname,0x3a,pn_pass,0x3C7230783E),0,0,0/**/FROM/**/md_users/**/WHERE/**/pn_uid=$id/* HTTP/1.1 Content-Length: 36 Content-Type: application/x-www-form-urlencoded Host: www.test.go.kr Connection: Keep-Alive User-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None).

Assistant: Since the input payload has union~all~select~from phrase, it is a type of SQL Injection attack.
